In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List
import numpy as np

# Конфигурация
API_BASE_URL = "http://localhost:8000"

print("=== ДЕМОНСТРАЦИЯ REST API ДЛЯ ПРЕДСКАЗАНИЯ ОТТОКА ===")

# 1. Проверка здоровья API
def check_health():
    """Проверка здоровья API"""
    try:
        response = requests.get(f"{API_BASE_URL}/health")
        if response.status_code == 200:
            health_data = response.json()
            print("✅ API здоров:")
            print(f"   Статус: {health_data['status']}")
            print(f"   Модель загружена: {health_data['model_loaded']}")
            print(f"   Версия: {health_data['version']}")
            return True
        else:
            print("❌ API не отвечает")
            return False
    except Exception as e:
        print(f"❌ Ошибка подключения к API: {e}")
        return False

# Проверяем здоровье
if not check_health():
    print("Запустите API командой: uvicorn api.main:app --reload")
    exit()

# 2. Получение информации о модели
def get_model_info():
    """Получение информации о модели"""
    response = requests.get(f"{API_BASE_URL}/model/info")
    if response.status_code == 200:
        model_info = response.json()
        print("\n📊 ИНФОРМАЦИЯ О МОДЕЛИ:")
        print(f"   Название: {model_info['model_name']}")
        print(f"   Версия: {model_info['model_version']}")
        print(f"   Тип: {model_info['model_type']}")
        print(f"   Количество признаков: {model_info['feature_count']}")
        print(f"   Дата обучения: {model_info['training_date']}")
        
        if model_info['performance']:
            print("   Метрики производительности:")
            for metric, value in model_info['performance'].items():
                print(f"     - {metric}: {value:.4f}")
        
        return model_info
    else:
        print("❌ Ошибка получения информации о модели")
        return None

model_info = get_model_info()

# 3. Пример данных для предсказания (только обязательные поля)
sample_customer = {
    "Tenure": 12.0,
    "CityTier": 2.0,
    "WarehouseToHome": 8.0,
    "HourSpendOnApp": 3.5,
    "NumberOfDeviceRegistered": 3.0,
    "SatisfactionScore": 4.0,
    "NumberOfAddress": 5.0,
    "Complain": 0.0,
    "OrderAmountHikeFromlastYear": 15.0,
    "CouponUsed": 4.0,
    "OrderCount": 25.0,
    "DaySinceLastOrder": 3.0,
    "CashbackAmount": 250.0
}

print(f"\n🎯 ПРИМЕР ДАННЫХ КЛИЕНТА (обязательные признаки):")
for feature, value in sample_customer.items():
    print(f"   {feature}: {value}")

# 4. Одиночное предсказание
def predict_single(customer_data: Dict, customer_id: int = None, threshold: float = 0.5):
    """Одиночное предсказание"""
    params = {"threshold": threshold}
    if customer_id:
        params["customer_id"] = customer_id
    
    response = requests.post(
        f"{API_BASE_URL}/predict",
        json=customer_data,
        params=params
    )
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Ошибка предсказания: {response.text}")
        return None

print("\n🔮 ОДИНОЧНОЕ ПРЕДСКАЗАНИЕ:")
prediction = predict_single(sample_customer, customer_id=50001, threshold=0.5)

if prediction:
    print(f"   ID клиента: {prediction['customer_id']}")
    print(f"   Вероятность оттока: {prediction['churn_probability']:.4f}")
    print(f"   Предсказание: {'ОТТОК' if prediction['prediction'] else 'НЕТ ОТТОКА'}")
    print(f"   Порог: {prediction['threshold']}")
    print(f"   Версия модели: {prediction['model_version']}")
    print(f"   Время предсказания: {prediction['timestamp']}")

# 5. Пакетное предсказание
def predict_batch(customers_data: List[Dict], customer_ids: List[int] = None, threshold: float = 0.5):
    """Пакетное предсказание"""
    request_data = {"customers": customers_data}
    if customer_ids:
        request_data["customer_ids"] = customer_ids
    
    response = requests.post(
        f"{API_BASE_URL}/predict/batch",
        json=request_data,
        params={"threshold": threshold}
    )
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Ошибка пакетного предсказания: {response.text}")
        return None

# Создаем несколько тестовых клиентов
batch_customers = [
    {
        "Tenure": 24.0, "CityTier": 1.0, "WarehouseToHome": 5.0,
        "HourSpendOnApp": 2.0, "NumberOfDeviceRegistered": 2.0,
        "SatisfactionScore": 5.0, "NumberOfAddress": 3.0, "Complain": 0.0,
        "OrderAmountHikeFromlastYear": 10.0, "CouponUsed": 8.0,
        "OrderCount": 40.0, "DaySinceLastOrder": 1.0, "CashbackAmount": 400.0
    },
    {
        "Tenure": 3.0, "CityTier": 3.0, "WarehouseToHome": 20.0,
        "HourSpendOnApp": 1.0, "NumberOfDeviceRegistered": 1.0,
        "SatisfactionScore": 2.0, "NumberOfAddress": 10.0, "Complain": 1.0,
        "OrderAmountHikeFromlastYear": 25.0, "CouponUsed": 1.0,
        "OrderCount": 5.0, "DaySinceLastOrder": 15.0, "CashbackAmount": 50.0
    },
    sample_customer  # Добавляем нашего первого клиента
]

batch_ids = [50002, 50003, 50004]

print("\n📦 ПАКЕТНОЕ ПРЕДСКАЗАНИЕ:")
batch_prediction = predict_batch(batch_customers, batch_ids, threshold=0.5)

if batch_prediction:
    print(f"   Всего клиентов: {batch_prediction['total_customers']}")
    print(f"   Процент оттока в батче: {batch_prediction['churn_rate']:.2%}")
    print(f"   Средняя вероятность: {batch_prediction['avg_probability']:.4f}")
    
    print("\n   Результаты предсказаний:")
    for i, pred in enumerate(batch_prediction['predictions']):
        status = "ОТТОК" if pred['prediction'] else "НЕТ ОТТОКА"
        print(f"     Клиент {pred['customer_id']}: {pred['churn_probability']:.4f} - {status}")

# 6. Анализ влияния threshold
print("\n📈 АНАЛИЗ ВЛИЯНИЯ THRESHOLD:")

thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
results = []

for threshold in thresholds:
    pred = predict_single(sample_customer, threshold=threshold)
    if pred:
        results.append({
            'threshold': threshold,
            'probability': pred['churn_probability'],
            'prediction': pred['prediction']
        })

if results:
    results_df = pd.DataFrame(results)
    print("Результаты предсказаний с разными порогами:")
    print(results_df.to_string(index=False))

# 8. Получение списка признаков
def get_feature_list():
    """Получение списка признаков"""
    response = requests.get(f"{API_BASE_URL}/features")
    if response.status_code == 200:
        return response.json()
    else:
        print("❌ Ошибка получения списка признаков")
        return None

print("\n📋 СПИСОК ПРИЗНАКОВ МОДЕЛИ:")
features_info = get_feature_list()
if features_info:
    print(f"   Всего признаков: {features_info['total_features']}")
    print(f"   Обязательных признаков: {len(features_info['required_features'])}")
    print(f"   Опциональных признаков: {len(features_info['optional_features'])}")
    
    print("\n   Обязательные признаки:")
    for feature in features_info['required_features']:
        print(f"     - {feature}")
    
    print("\n   Опциональные признаки:")
    for feature in features_info['optional_features']:
        print(f"     - {feature}")
    
    print("\n   Все признаки модели:")
    for feature in features_info['features']:
        print(f"     - {feature}")

# 9. Тестирование валидации
print("\n🧪 ТЕСТИРОВАНИЕ ВАЛИДАЦИИ:")

# Невалидные данные - SatisfactionScore вне диапазона
invalid_customer = sample_customer.copy()
invalid_customer["SatisfactionScore"] = 10.0  # Невалидное значение

response = requests.post(f"{API_BASE_URL}/predict", json=invalid_customer)
if response.status_code == 400:
    print("✅ Валидация работает: невалидный SatisfactionScore отклонен")
    print(f"   Сообщение об ошибке: {response.json()['detail']}")
else:
    print("❌ Валидация не сработала")

# Неполные данные
incomplete_customer = {"Tenure": 12.5}  # Только одно поле

response = requests.post(f"{API_BASE_URL}/predict", json=incomplete_customer)
if response.status_code == 422:
    print("✅ Валидация работает: неполные данные отклонены")
    print(f"   Сообщение об ошибке: {response.json()['detail']}")
else:
    print("❌ Валидация не сработала")

# Проверка работы с несуществующими эндпоинтами
response = requests.get(f"{API_BASE_URL}/features/list")  # Старый эндпоинт
if response.status_code == 404:
    print("✅ Старый эндпоинт /features/list не существует")
else:
    print(f"⚠ Неожиданный ответ: {response.status_code}")

# 10. Получение версии API
print("\n📦 ИНФОРМАЦИЯ О ВЕРСИИ:")
response = requests.get(f"{API_BASE_URL}/version")
if response.status_code == 200:
    version_info = response.json()
    print(f"   Версия API: {version_info['api_version']}")
    print(f"   Версия модели: {version_info['model_version']}")
    print(f"   Время: {version_info['timestamp']}")

# 11. Сводная таблица предсказаний
print("\n📊 СВОДНАЯ ТАБЛИЦА ПРЕДСКАЗАНИЙ:")

# Создаем DataFrame с результатами
if batch_prediction and 'predictions' in batch_prediction and batch_prediction['predictions']:
    summary_df = pd.DataFrame(batch_prediction['predictions'])
    
    # Добавляем признаки
    for i, customer_data in enumerate(batch_customers):
        if i < len(summary_df):
            for feature, value in customer_data.items():
                summary_df.at[i, feature] = value
    
    print("\nДетализированные результаты:")
    print(summary_df[['customer_id', 'Tenure', 'SatisfactionScore', 'Complain', 
                      'churn_probability', 'prediction']].to_string(index=False))
    
    # Статистика
    print("\nСтатистика по предсказаниям:")
    print(f"   Средняя вероятность оттока: {summary_df['churn_probability'].mean():.4f}")
    print(f"   Медианная вероятность оттока: {summary_df['churn_probability'].median():.4f}")
    print(f"   Минимальная вероятность: {summary_df['churn_probability'].min():.4f}")
    print(f"   Максимальная вероятность: {summary_df['churn_probability'].max():.4f}")
    print(f"   Стандартное отклонение: {summary_df['churn_probability'].std():.4f}")
    
    # Корреляция между признаками и вероятностью оттока
    print("\nКорреляция признаков с вероятностью оттока:")
    numeric_features = summary_df.select_dtypes(include=[float, int]).columns
    for feature in numeric_features:
        if feature != 'churn_probability' and feature != 'customer_id':
            correlation = summary_df[feature].corr(summary_df['churn_probability'])
            print(f"   {feature}: {correlation:.4f}")
else:
    print("⚠ Нет данных для создания сводной таблицы")

print("\n🎉 ДЕМОНСТРАЦИЯ ЗАВЕРШЕНА!")
print("\nДоступные эндпоинты:")
print("  GET  /               - Корневой эндпоинт")
print("  GET  /health         - Проверка жизнеспосбности API")
print("  GET  /model/info     - Информация о модели")
print("  POST /predict        - Одиночное предсказание")
print("  POST /predict/batch  - Пакетное предсказание")
print("  GET  /features       - Список признаков модели")
print("  GET  /version        - Версия API")

print(f"\n📚 Документация API:")
print(f"  Swagger UI: {API_BASE_URL}/docs")
print(f"  ReDoc: {API_BASE_URL}/redoc")
print(f"  OpenAPI спецификация: {API_BASE_URL}/openapi.json")

=== ДЕМОНСТРАЦИЯ REST API ДЛЯ ПРЕДСКАЗАНИЯ ОТТОКА ===
✅ API здоров:
   Статус: healthy
   Модель загружена: True
   Версия: 1.0.0

📊 ИНФОРМАЦИЯ О МОДЕЛИ:
   Название: Предиктор оттоков клиентов
   Версия: 1.0.0
   Тип: LGBMClassifier
   Количество признаков: 37
   Дата обучения: unknown

🎯 ПРИМЕР ДАННЫХ КЛИЕНТА (обязательные признаки):
   Tenure: 12.0
   CityTier: 2.0
   WarehouseToHome: 8.0
   HourSpendOnApp: 3.5
   NumberOfDeviceRegistered: 3.0
   SatisfactionScore: 4.0
   NumberOfAddress: 5.0
   Complain: 0.0
   OrderAmountHikeFromlastYear: 15.0
   CouponUsed: 4.0
   OrderCount: 25.0
   DaySinceLastOrder: 3.0
   CashbackAmount: 250.0

🔮 ОДИНОЧНОЕ ПРЕДСКАЗАНИЕ:
   ID клиента: 50001
   Вероятность оттока: 0.0000
   Предсказание: НЕТ ОТТОКА
   Порог: 0.5
   Версия модели: 1.0.0
   Время предсказания: 2025-12-06T22:55:58.580042

📦 ПАКЕТНОЕ ПРЕДСКАЗАНИЕ:
   Всего клиентов: 3
   Процент оттока в батче: 0.00%
   Средняя вероятность: 0.0116

   Результаты предсказаний:
     Клиент 50002: 0.

c:\Users\Артем\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Артем\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
